In [6]:
import palom
import numpy as np

In [11]:
def run_palom(
    img_paths,
    img_modes,
    pixel_size,
    channel_names,
    output_path,
    qc_path,
    level
):
    ref_reader = reader.SvsReader(img_paths[0])
    ref_color_proc = color.PyramidHaxProcessor(ref_reader.pyramid)
    ref_thumbnail_level = ref_reader.get_thumbnail_level_of_size(2500)

    block_affines = []
    for idx, p in enumerate(img_paths[1:]):
        logger.info(f"Processing {p.name}")
        if p == img_paths[0]:
            block_affines.append(np.eye(3))
            continue
        moving_reader = reader.SvsReader(p)
        moving_color_proc = color.PyramidHaxProcessor(moving_reader.pyramid)
        moving_thumbnail_level = moving_reader.get_thumbnail_level_of_size(2500)

        aligner = align.Aligner(
            ref_color_proc.get_processed_color(level, 'grayscale'),
            moving_color_proc.get_processed_color(level, 'grayscale'),
            ref_color_proc.get_processed_color(ref_thumbnail_level, 'grayscale').compute(),
            moving_color_proc.get_processed_color(moving_thumbnail_level, 'grayscale').compute(),
            ref_reader.level_downsamples[ref_thumbnail_level] / ref_reader.level_downsamples[level],
            moving_reader.level_downsamples[moving_thumbnail_level] / moving_reader.level_downsamples[level]
        )

        aligner.coarse_register_affine()
        
        # FIXME move the saving figure logic 
        plt.suptitle(f"L: {ref_reader.path.name}\nR: {p.name}")
        fig_w = max(plt.gca().get_xlim())
        fig_h = max(plt.gca().get_ylim()) + 100
        factor = 1600 / max(fig_w, fig_h)
        plt.gcf().set_size_inches(fig_w*factor/72, fig_h*factor/72)
        plt.tight_layout()
        plt.savefig(qc_path / f"{idx+1:02d}-{p.name}.png", dpi=72)
        plt.close()
        
        aligner.compute_shifts()
        aligner.constrain_shifts()

        block_affines.append(aligner.block_affine_matrices_da)
    
    mosaics = []
    m_ref = ref_color_proc.get_processed_color(level=level, mode=img_modes[0])
    mosaics.append(m_ref)
    for p, m, mx in zip(img_paths[1:], img_modes[1:], block_affines):
        moving_color_proc = color.PyramidHaxProcessor(
            reader.SvsReader(p).pyramid
        )
        m_moving = align.block_affine_transformed_moving_img(
            ref_color_proc.get_processed_color(level),
            moving_color_proc.get_processed_color(level, mode=m),
            mx
        )
        mosaics.append(m_moving)

    if pixel_size is None:
        pixel_size = ref_reader.pixel_size

    pyramid.write_pyramid(
        pyramid.normalize_mosaics(mosaics),
        output_path,
        pixel_size=pixel_size,
        channel_names=channel_names
    )
    return 0


In [12]:
run_palom_cycif(
    img_paths = [
        "./../../../../data/Human_squamous_cell_carcinoma_stained_with_SignalStar_mIHC_technology/numpy_stacked_output_uint16.tif"
    ],
    pixel_size = 0.49884663906387205,
    channel_names = ['Alexa 488', 'Alexa 594', 'Alexa 647', 'Cy7', 'DAPI', 'Autofluorescence'],
    output_path = r"./../../../../data/Human_squamous_cell_carcinoma_stained_with_SignalStar_mIHC_technology/palom/mosaic.ome.tiff",
    level = 0
)

IndexError: list index out of range